# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Print the current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace/UdacityDataEngineeringNanodegree/Project_1B_Data_Modelling_in_Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# Create a list for each data row for all the files present in the file_path_list
for f in file_path_list:

    # Reading CSV files 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        
        # Creating a CSV reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # Extracting each data row one by one and append it
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Print the total number of rows 
# print(len(full_data_rows_list))

# Print the contents of event data rows
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# Check the number of rows in event_datafile_new.csv file
print("The number of lines rows in event_datafile_new.csv file is: ")
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

The number of lines rows in event_datafile_new.csv file is: 
6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

In [5]:
def execute_query(session, query, parameter_list=[]):
    '''
    Executing the given query and prints exception in case of errors.
    '''
    try:
        if(parameter_list == []):
            # Return values in case of Select (and Create) query
            return session.execute(query)
        else:
            session.execute(query, parameter_list)
    except Exception as e:
        print(e)

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()

# Create a session variable to establish connection and execute queries
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 

create_keyspace_query = """
CREATE KEYSPACE IF NOT EXISTS udacity
WITH REPLICATION = 
{'class': 'SimpleStrategy',
'replication_factor': 1}
"""

execute_query(session, create_keyspace_query)

#### Set Keyspace

In [8]:
# Set the session keyspace to the newly created keyspace

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Creating tables in Apache Cassandra to answer the 3 business queries requried.

## Query 1

### 1. Give the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4


### Rationale behind the Primary Key Selection:
A sessionId consists of one or more itemInSession values. 

The query 1 focusses on the sessionId and drilling down to a particular itemInSession value corresponding to the sessionId. 

From this, we can understand that the sessionId and itemInSession pair is unique for a row. 
Also, we would like to filter the rows based on both, the sessionId and the itemInSession. 

So the Partition key = (sessionId, itemInSession) is a good choice. 

#### Create a table, music_session_info, to cater the needs of Query 1


In [9]:
create_query_1 = """
CREATE TABLE IF NOT EXISTS music_session_info
(sessionId INT, 
itemInSession INT,
artist TEXT, 
song TEXT, 
length FLOAT, 
PRIMARY KEY ((sessionId, itemInSession)) )
"""

execute_query(session, create_query_1) ;

#### Insert data into music_session_info table from event_datafile_new.csv


In [10]:
file = 'event_datafile_new.csv'


# Insert query for inserting data into the table
insert_query_1 = "INSERT INTO music_session_info \
                (sessionId, itemInSession, artist, \
                song, length) "
insert_query_1 = insert_query_1 + "VALUES (%s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # Skip header
    next(csvreader) 
    
    for line in csvreader:
        # Value list for insert query
        parameter_list = [int(line[8]), int(line[3]), line[0], \
                          line[9], float(line[5])]
        
        execute_query(session, insert_query_1, parameter_list)


#### Verify the data by executing the query for music_session_info table

In [11]:
select_query_1 = """
SELECT artist, song, length
FROM music_session_info
WHERE sessionId = 338 AND itemInSession = 4
"""

pd.DataFrame(list(execute_query(session, select_query_1)))

artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312

## Query 2

### Give the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


### Rationale behind the Primary Key Selection:
In Query 2, we require information for a particular user and session. Also, we want this to be ordered by itemInSession. 

A user can have multiple sessions, and each of these sessions can have multiple itemInSession values. 
So the triplet, userId, sessionId and itemInSession can be used to uniquely identify a row.

We want the data for a particular userId and sessionId to be present on a single node, so that we can filter the rows based on the userId and sessionId, ordered by itemInSession. 

Hence, it is a good choice to have partitionKey = (userId, sessionId) and clusteringKey = itemInSession

#### Create a table, user_preference_info, to cater the needs of Query 1


In [12]:
create_query_2 = """
CREATE TABLE IF NOT EXISTS user_preference_info
(userId INT,
sessionId INT, 
itemInSession INT,
artist TEXT,
song TEXT, 
userFirstName TEXT,
userLastName TEXT,
PRIMARY KEY ((userId, sessionId), itemInSession))
"""

execute_query(session, create_query_2);

#### Insert data into user_preference_info table from event_datafile_new.csv


In [13]:
file = 'event_datafile_new.csv'

# Insert query for inserting data into the table
insert_query_2 = "INSERT INTO user_preference_info (\
                userId, sessionId, itemInSession, \
                artist, song, userFirstName, \
                userLastName)"
        
insert_query_2 = insert_query_2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        # Value list for insert query
        parameter_list = (int(line[10]), int(line[8]), int(line[3]), \
                          line[0], line[9], line[1], \
                          line[4])
        
        execute_query(session, insert_query_2, parameter_list)


#### Verify the data by executing the query for user_preference_info table

In [14]:
select_query_2 = """
SELECT artist, song, userFirstName, 
userLastName
FROM user_preference_info
WHERE userId = 10 AND sessionId = 182
"""

pd.DataFrame(list(execute_query(session, select_query_2)))

artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  userfirstname userlastname  
0        Sylvie         Cruz  
1        Sylvie         Cruz  
2        Sylvie         Cruz  
3        Sylvie         Cruz

---
### Query 3

### Give every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

### Rationale behind the Primary Key Selection:
In Query 3, we require user name information for a particular song. So, each row can be uniquely identified using a song and a user. 

Since we filter the records based on song alone and not on userId, our partitionKey = song and our clusteringKey = userId

In [15]:
create_query_3 = """
CREATE TABLE IF NOT EXISTS song_listeners_info
(song TEXT,
userId INT, 
userFirstName TEXT,
userLastName TEXT,
PRIMARY KEY (song, userId) )
"""

execute_query(session, create_query_3);

In [16]:
file = 'event_datafile_new.csv'

insert_query_3 = "INSERT INTO song_listeners_info (\
                song, userId, userFirstName, \
                userLastName)"
        
insert_query_3 = insert_query_3 + "VALUES (%s, %s, %s, %s)"
        
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    # Skip header
    next(csvreader) 
    for line in csvreader:

        parameter_list = [line[9], int(line[10]), line[1], \
                          line[4]]
        execute_query(session, insert_query_3, parameter_list)

In [17]:
select_query_3 = """
SELECT userFirstName, userLastName \
FROM song_listeners_info
WHERE song = 'All Hands Against His Own'
"""

pd.DataFrame(list(execute_query(session, select_query_3)))


userfirstname userlastname
0    Jacqueline        Lynch
1         Tegan       Levine
2          Sara      Johnson

### Drop the tables before closing out the sessions

In [18]:
drop_query_1 = """
DROP TABLE music_session_info
"""

execute_query(session, drop_query_1)

drop_query_2 = """
DROP TABLE user_preference_info
"""

execute_query(session, drop_query_2)

drop_query_3 = """
DROP TABLE song_listeners_info
"""

execute_query(session, drop_query_3) ;

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()